In [3]:
import os
import re
import json
import jsonlines as jl
import joblib
from pathlib import Path
import itertools
from collections import defaultdict

import pickle

import numpy as np
from numpy.linalg import norm

from tqdm import tqdm

from scipy.stats import spearmanr, pearsonr
from sklearn.model_selection import train_test_split

import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.distributed import DistributedSampler

from transformers import get_scheduler

from keras.preprocessing.text import Tokenizer

# Useful Constants

In [4]:
#################
# Paths and files
#################

# Data paths
INPUT_PATH = Path("..")/"input"
ALL_TRIPLES = Path(INPUT_PATH)/"triple_set.jsonl"
TRAIN_TRIPLES = Path(INPUT_PATH)/"train_triples.jsonl"
VAL_TRIPLES = Path(INPUT_PATH)/"val_triples.jsonl"
TEST_TRIPLES = Path(INPUT_PATH)/"test_triples.jsonl"

# Model paths
OUTPUT_PATH = Path("..")/"result"
FASTTEXT = "./ft_all_phr_2022_30_03_top_100000.pkl"
FIT_TOKENIZER = Path(OUTPUT_PATH)/"ft_tokenizer.pkl"
OUTPUT_MODEL = Path(OUTPUT_PATH)/"triplet_loss_model.sav"

###########
# Сonstants
###########

# Data fields
ANCHOR = "anchor"
POSITIVE = "positive"
NEGATIVE = "negative"
TYPE2IDX = dict(anchor=0, positive=1, negative=2)

# Devices and threads
GPU_NUM = torch.cuda.device_count()
GPU_IDS = [f'cuda:{_id}' for _id in range(GPU_NUM)]

# Model parameters
VOCAB_SIZE = 20000
EMBEDDING_SIZE = 300
LSTM_SIZE = 128
LSTM_NUM_LAYERS = 3
OUT_EMBEDDING_SIZE = 768
TRAIN_BATCH_SIZE = 1024
VAL_BATCH_SIZE = 1024
MAX_SEQ_LEN = 32

# Loading Data

In [5]:
def load_triples(_file):
    with jl.open(_file, mode="r") as infile:
        triple_lst = [item for item in infile]
    return triple_lst

In [6]:
with open(FIT_TOKENIZER, "rb") as infile:
    TOKENIZER = pickle.load(infile)

In [7]:
NB_WORDS = min(VOCAB_SIZE, len(TOKENIZER.word_index.items()))
NB_WORDS

20000

In [10]:
dataset_is_split = os.path.exists(TRAIN_TRIPLES) and os.path.exists(VAL_TRIPLES) and os.path.exists(TEST_TRIPLES)

if not dataset_is_split:
    # Split data
    all_triples = load_triples(ALL_TRIPLES)
    train_triples, val_test_triples = train_test_split(all_triples, test_size=0.3, random_state=1)
    val_triples, test_triples = train_test_split(val_test_triples, test_size=0.33, random_state=1)
    # Dump sets
    with jl.open(TRAIN_TRIPLES, mode="w") as outfile:
        outfile.write_all(train_triples)
    with jl.open(VAL_TRIPLES, mode="w") as outfile:
        outfile.write_all(val_triples)
    with jl.open(TEST_TRIPLES, mode="w") as outfile:
        outfile.write_all(test_triples)
else:
    train_triples = load_triples(TRAIN_TRIPLES)
    val_triples = load_triples(VAL_TRIPLES)

False


# FastText + BiLSTM Training

In [11]:
# Utilities

class _dict(dict):
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    
    def __getstate__(self):
        return self.__dict__
    
    def __setstate__(self, d):
        return self.__dict__.update(d)

In [12]:
class TextPreprocessor:
    def __init__(self, args=None, tokenizer=TOKENIZER, pretrained_embeddings=FASTTEXT):
        if args is not None:
            self.args = vars(args)
        if not args:
            self.args = _dict(tokenizer=tokenizer,
                             pretrained_embeddings=joblib.load(pretrained_embeddings))
            
    def to_matrix(self, data, _dtype=torch.int64, batch_first=True):
        indexed_data = [[], [], []]
        
        # Texts to token IDs
        for item in data:
            indexed_data[0].append(self.clean_text(item[ANCHOR]))
            indexed_data[1].append(self.clean_text(item[POSITIVE]))
            indexed_data[2].append(self.clean_text(item[NEGATIVE]))
        indexed_data = list(map(self.args.tokenizer.texts_to_sequences, indexed_data))
        
        # Sequence length dimension size
        max_seq_len = MAX_SEQ_LEN
        
        # Filling the padded matrix
        _size = len(data)
        out_matrix = torch.zeros((_size, 3, max_seq_len), dtype=_dtype)
        for idx in range(_size):
            for item_type in range(3):
                bound = min(len(indexed_data[item_type][idx]), max_seq_len)
                out_matrix[idx, item_type, :bound] = torch.from_numpy(
                    np.array(indexed_data[item_type][idx][:bound]))
        if not batch_first:
            out_matrix = out_matrix.permute(1, 2, 0)
            
        return out_matrix
    
    def build_embedding_matrix(self, emb_size=EMBEDDING_SIZE, nb_words=NB_WORDS, _dtype=torch.float32):
        embedding_matrix = torch.zeros((nb_words + 1, emb_size), dtype=_dtype)
        oof_tokens = set()
        
        for word, i in self.args.tokenizer.word_index.items():
            if i >= VOCAB_SIZE + 1:
                continue
            embedding_vector = None
            try:
                embedding_vector = self.args.pretrained_embeddings[word] 
            except:
                oof_tokens.add(word)
            if embedding_vector is not None:
                embedding_matrix[i, :] = torch.from_numpy(embedding_vector)
                
        print(f"{len(oof_tokens)} out of FastText tokens")
        return embedding_matrix
    
    @staticmethod
    def clean_text(_text):
        _text = re.sub("\n|(</?[^>]*>)", " ", _text)
        _text = re.sub("\.\s", " ", _text)
        _text = re.sub('здравствуйте', '', _text)
        _text = re.sub('добрый день', '', _text)
        _text = re.sub(r"\s+", " ", _text)
        return _text.strip()

In [13]:
text_preprocessor = TextPreprocessor()
embedding_matrix = text_preprocessor.build_embedding_matrix()
embedding_matrix.shape

3153 out of FastText tokens


torch.Size([20001, 300])

In [14]:
class TripletSet:
    def __init__(self, train_data, val_data,
                 preprocessor=text_preprocessor, batch_size=(TRAIN_BATCH_SIZE, VAL_BATCH_SIZE)):
        kwargs = self.make_kwargs()
        
        train_features = preprocessor.to_matrix(train_data)
        train_labels = torch.ones(len(train_data))
        val_features = preprocessor.to_matrix(val_data)
        val_labels = torch.ones(len(val_data))
        
        self.train_set = TensorDataset(train_features, train_labels)
        self.train_loader = DataLoader(self.train_set, batch_size=batch_size[0],
                                       drop_last=True, shuffle=True, **kwargs)
        self.val_set = TensorDataset(val_features, val_labels)
        self.val_loader = DataLoader(self.val_set, batch_size=batch_size[1],
                                     drop_last=True, shuffle=True, **kwargs)
        
    @staticmethod
    def make_kwargs():
        use_cuda = torch.cuda.is_available()
        print(f"Use CUDA {use_cuda}")
        kwargs = {'num_workers': 4 * GPU_NUM, 'pin_memory': True} if use_cuda else {}
        return kwargs

In [15]:
_dataset = TripletSet(train_triples, val_triples)

Use CUDA True


In [16]:
class SoftmaxLoss(nn.Module):
    def __init__(self, _loss=(lambda _: F.relu(_))):
        super(SoftmaxLoss, self).__init__()
        self.loss_func = _loss
        
    def forward(self, anchor, positive, negative, is_test=False):
        positive_similarity = torch.sum(anchor * positive, axis=-1, keepdims=True)
        _matmul = torch.matmul(anchor, torch.transpose(negative, 0, 1))
        negative_similarity = torch.log(torch.sum(
            torch.exp(_matmul), axis=-1, keepdims=True))
        loss = self.loss_func(negative_similarity - positive_similarity)
        return loss
    
    @staticmethod
    def mean_loss(_true, _predicted):
        _mean = torch.mean(_predicted - 0 * _true, dim=0)
        return _mean

In [17]:
class TripletLossModel(nn.Module):
    def __init__(self, criterion,
                 init_weights=embedding_matrix,
                 emb_size=EMBEDDING_SIZE, vocab_size=VOCAB_SIZE,
                 lstm_num_layers=LSTM_NUM_LAYERS,
                 lstm_hidden_size=LSTM_SIZE, is_bi=True,
                 out_emb_size=OUT_EMBEDDING_SIZE,
                 batch_first=True, dropout=0.2, norm_eps=1e-12):
        super(TripletLossModel, self).__init__()
        
        # Parameters
        self.emb_size = emb_size
        self.vocab_size = vocab_size
        self.lstm_num_layers = lstm_num_layers
        self.lstm_hidden_size = lstm_hidden_size
        self.directions = int(is_bi) + 1
        self.out_emb_size = out_emb_size
        
        # Layers
        self.embedding = nn.Embedding(self.vocab_size + 1, self.emb_size)
        self.embedding.weight = nn.Parameter(init_weights)
        self.lstm = nn.LSTM(input_size=self.emb_size, hidden_size=self.lstm_hidden_size,
                            num_layers=self.lstm_num_layers, bidirectional=is_bi,
                            batch_first=batch_first, dropout=dropout)
        self.cosine_similarity = nn.CosineSimilarity(dim=1, eps=1e-6)
        
        self.best = defaultdict(int)
        
    def forward(self, _input):
        anchor_input = self.fetch_triplet_item(_input, ANCHOR)
        positive_input = self.fetch_triplet_item(_input, POSITIVE)
        negative_input = self.fetch_triplet_item(_input, NEGATIVE)
        
        encoded_anchor = self.encode(anchor_input)
        encoded_positive = self.encode(positive_input)
        encoded_negative = self.encode(negative_input)
        
        cosine_pos = self.cos(encoded_anchor, encoded_positive)
        cosine_neg = self.cos(encoded_anchor, encoded_negative)
        
        return _dict(anchor=encoded_anchor,
                    positive=encoded_positive,
                    negative=encoded_negative,
                    cos_sim=(cosine_pos, cosine_neg))
        
    def encode(self, tokenized_seq, _const=0.5):
        encoded_seq = self.embedding(tokenized_seq)
        encoded_seq, _ = self.lstm(encoded_seq)
        encoded_seq = encoded_seq[:, -1, :] * _const
        return encoded_seq
        
    @staticmethod
    def fetch_triplet_item(_input, item_type):
        if TYPE2IDX.get(item_type) is None:
            raise Exception(f"No item type {item_type}")
        return _input[:, TYPE2IDX[item_type], :]
    
    def cos(self, u, v):
        return self.cosine_similarity(u, v)

In [18]:
class ModelPipeline:
    def __init__(self, dataset, model, criterion, optimizer, device):
        self.dataset = dataset
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.device = torch.device(device)
        self.lr_scheduler = None
        
        self.best = defaultdict(int)
        self.progress_bar = None
        
    def fit(self, epoch_num, log_interval=10):
        if self.device.type == 'cuda':
            self.model.cuda()
        
        num_steps = epoch_num * len(self.dataset.train_loader)
        
        self.progress_bar = tqdm(range(num_steps), position=0, leave=True)
        self.lr_scheduler = get_scheduler(name="linear", optimizer=self.optimizer,
                                    num_warmup_steps=0, num_training_steps=num_steps)
        for epoch in range(epoch_num):
            self.train_epoch(epoch)
            
            if (epoch + 1) % log_interval == 0 or epoch == 0:
                print()
                self.validate()
                print()
    
    def train_epoch(self, epoch):
        self.model.train()
        
        epoch_loss = []
        
        for _input, _target in self.dataset.train_loader:
            if self.device.type == 'cuda':
                _input, _target = _input.cuda(), _target.cuda()
            
            _output = self.model(_input)
            loss = self.criterion(_output.anchor, _output.positive, _output.negative)
            mean_loss = self.criterion.mean_loss(loss, loss)
            epoch_loss.append(mean_loss.cpu().item())
            
            mean_loss.backward()
            self.optimizer.step()
            self.lr_scheduler.step()
            self.optimizer.zero_grad()
            
            self.progress_bar.update(1)
            
        print(f"Epoch #{epoch + 1}\nMean epoch loss: {sum(epoch_loss) / len(epoch_loss)}")
    
    def validate(self):
        self.model.eval()
        
        with torch.no_grad():
            cosine_dist, targets = [], []
            
            for _input, _target in self.dataset.val_loader:
                if self.device.type == "cuda":
                    _input, _target = _input.cuda(), _target.cuda()
                
                _output = self.model(_input)
                cosine_dist.append(_output.cos_sim[0])
                cosine_dist.append(_output.cos_sim[1])
                targets.append(_target)
                targets.append(_target * -1)
                
            cosine_dist = torch.cat(cosine_dist).cpu().detach().numpy()
            targets = torch.cat(targets)
            targets = torch.squeeze(targets).cpu().detach().numpy()
        
        for metric, func in [("spearman_r", spearmanr), ("pearson_r", pearsonr)]:
            coef, _ = func(targets, cosine_dist)
            coef = np.round(coef, 4)

            metric_name = f"{metric}"
            message = f"{metric} = {coef}"
            if coef > self.best[metric_name]:
                self.best[metric_name] = coef
                message = "*** New best: " + message
                if metric == "spearman_r":
                    torch.save(self.model.state_dict(), OUTPUT_MODEL)

            print(message)

In [19]:
_model = TripletLossModel(embedding_matrix)
_criterion = SoftmaxLoss()
_optimizer = Adam(_model.parameters(), lr=1e-5)
_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

evaluation_pipeline = ModelPipeline(dataset=_dataset, model=_model, criterion=_criterion,
                                    optimizer=_optimizer, device=_device)

In [20]:
evaluation_pipeline.fit(epoch_num=150)

  1%|          | 683/102450 [01:28<3:29:12,  8.11it/s]

Epoch #1
Mean epoch loss: 6.61628636154912

*** New best: spearman_r = 0.6213
*** New best: pearson_r = 0.5877



  1%|▏         | 1366/102450 [03:08<3:27:43,  8.11it/s] 

Epoch #2
Mean epoch loss: 5.869832014444108


  2%|▏         | 2049/102450 [04:37<3:26:06,  8.12it/s] 

...

Epoch #148
Mean epoch loss: 0.3909447586728399


 99%|█████████▉| 101767/102450 [3:47:15<01:24,  8.10it/s]

Epoch #149
Mean epoch loss: 0.38955496345397145


100%|██████████| 102450/102450 [3:48:46<00:00,  8.09it/s]

Epoch #150
Mean epoch loss: 0.3899305074927049

spearman_r = 0.8641
*** New best: pearson_r = 0.9138

